## **TD 11-12**

### Collecte, Traitement, et Analyse de données de réseaux sociaux

LEFEVRE Laura et LE CORRE Camille - LDD BI

In [1]:
class tweet:
    def __init__(self) -> None:
        tweet = {}

In [ ]:
tweet1 = tweet()